<a href="https://colab.research.google.com/github/GURJEW/grammemer/blob/main/_dialog_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dialog Recognizer
Для распознавания разместите однокональные mp3-файлы в папку "mp3" и запустите ячейки ноутбука.

In [ ]:
%%capture
!pip install boto3
!pip install pyannote.audio
!pip install speechbrain

In [ ]:
import boto3
import json
import numpy as np
import os
import pandas as pd
import pickle
import pyannote.audio
import requests
import time
import torch
import torchaudio
import warnings

from IPython.display import clear_output, Audio
from collections import namedtuple
from google.colab import drive
from pyannote.audio.utils.signal import Binarize
from speechbrain.pretrained import SepformerSeparation
from tqdm.notebook import tqdm
from torch.nn import ReplicationPad1d, AvgPool1d
from torch.nn.functional import normalize
from torchaudio.transforms import Resample
from typing import Tuple


Wave = namedtuple('Wave', ['data', 'rate'])
WaveParams = namedtuple(
    'WaveParams', ['channels', 'length', 'duration', 'sample_rate']
    )

START_END = ['start', 'end']

warnings.filterwarnings('ignore')
path = '/content/gdrive/My Drive/Projects/'
project = 'DialogRecognizer/'
drive.mount('/content/gdrive')
os.chdir(path + project)
pd.set_option('display.max_colwidth', 120)
clear_output()

## Downloads
* Создание директорий в рабочем каталоге для хранения промежуточных и выходнях данных.
* Ввод имени эксперимента и создание соответствующих поддиректорий в созданных на предыдущем шаге.
* Загрузка имеющихся данных.


### defs

In [ ]:
def create_trial(trial_name: str=None) -> str:
    if trial_name is None:
        trial_name = input('Введите название эксперимента: ')
    make_dirs(trial_name)
    return trial_name


def get_wave_params(call_id: str) -> tuple:
    '''
    Определение параметров аудиозаписи.

    Аргументы:
    ::  call_id -- идентификатор аудиозаписи

    Возвращает именованный картеж параметров аудиозаписи:
    ::  num_chanels -- количество каналов
    ::  length -- длина тензора данных
    ::  duration -- длительность
    ::  sample_rate -- частота дискретизации
    '''
    if 'waves' in globals():
        data, sample_rate = waves[call_id]
    else:
        path = os.path.join(trial_name, 'wav', call_id)
        data, sample_rate = torchaudio.load(f'{path}.wav')
    channels = data.shape[0]
    length = data.shape[1]
    duration = data.shape[1] / sample_rate
    return WaveParams(channels, length, duration, sample_rate)


def exstract_call_ids(paths: list) -> list:
    return [os.path.split(path)[-1].split('.')[0] for path in paths]


def get_call_ids(paths: list) -> list:
    '''
    Получение идентификаторов аудиозаписей из названий файлов.

    Аргументы:
    ::  paths -- список путей к данным

    Возвращает список идентификаторов аудиозаписей.
    '''
    ids = set()
    for path in paths:
        if 'txt' in path:             
            call_ids = exstract_call_ids(read_links(path))            
        else:
            call_ids = exstract_call_ids(os.listdir(path))
        ids = ids.union(call_ids)
    return list(ids)


def load_segments(call_id: str, channels: int, segment_ids: list=None) -> Wave:
    '''
    Загрузка списка фрагментов аудиозаписей.

    Аргументы:
    ::  call_id -- идентификатор аудиозаписи
    ::  channels -- количество каналов аудиозаписи

    Возвращает картеж из списка фрагментов и частоты дискретизации.
    '''

    path = f'{trial_name}/segments/{channels}'
    if call_id in os.listdir(path):
        sample_rate = None
        segments = []
        if segment_ids is None:
            segment_ids = range(len(os.listdir(f'{path}/{call_id}')))
        for segment_id in segment_ids:
            data, rate = torchaudio.load(f'{path}/{call_id}/{segment_id}.wav')
            if sample_rate is None:
                sample_rate = rate
            assert sample_rate == rate, f'{sample_rate} != {rate}'
            segments.append(data)
        return Wave(segments, sample_rate)


def load_waves(ids: list=None) -> dict:
    '''
    Загрузка коллекции аудиофайлов.

    Аргументы:
    ::  ids -- список идентификаторов аудиофайлов

    Возвращает словарь {call_id: Wave}.

    '''
    if ids is None:
        assert 'call_ids' in globals(), 'список идентификаторов не найден'
        ids = call_ids
    
    
    return {
        call_id: Wave(*torchaudio.load(f'{trial_name}/wav/{call_id}.wav'))
        for call_id in tqdm(ids)
    }


def load_word_annotations(ids: list=None):
    '''
    Загрузка коллекции пословных аннотаций.

    Аргументы:
    ::  idx -- список идентификаторов аудиофайлов

    Возвращает словарь {call_id: pd.DataFrame}.

    '''
    if ids is None:
        assert 'call_ids' in globals(), 'список идентификаторов не найден'
        ids = call_ids
    return {
        call_id: pd.read_csv(f'{trial_name}/words/{call_id}.csv')
        for call_id in tqdm(ids)
    }


def make_dirs(trial_name: str) -> None:
    '''
    Создание директорий.
    '''
    def make(components) -> None:
        os.makedirs(os.path.join(*components), exist_ok=True)

    dirs = ['mp3', 'wav', 'segments', 'words', 'dialogs', 'diarization']
    for dir_name in dirs:
        if dir_name == 'segments':
            for channel in ('1', '2'):
                make([trial_name, dir_name, channel])
        else:
            make([trial_name, dir_name])   


def read_links(path: str='links.txt') -> list:
    '''
    Чтение списка ссылок на аудиофайлы.

    Аргументы:
    ::  path -- путь к файлу с сылками на аудиофайлы

    Возвращает список ссылок.
    '''
    with open(path) as f:
        links = [link.strip() for link in f.readlines()]
    return links


# def rename_trial(name: str, dirs: list=['mp3']) -> str:
#     for dir_name in dirs:
#         for dir_path, _, file_names in os.walk(dir_name):
#             for file_name in file_names:
#                 os.renames(
#                     os.path.join(dir_path, trial_name, file_name),
#                     os.path.join(dir_path, name, file_name)
#                 )
#     return name

### process

In [ ]:
trial_name = input('Введите название эксперимента: ')

Введите название эксперимента: stereo


In [ ]:
trial_name = create_trial('marginal')

In [ ]:
call_ids = ['mix_13054_16e013__2021_08_26__17_45_09_037']

In [ ]:
trial_name

'marginal'

## MP3 to WAV
Конвертация mp3-файлов в wav-формат.

### defs

In [ ]:
def converte(
    call_id_or_uri: str,
    source: str='mp3',
    target: str='wav',
    mono: bool=False
    ) -> None:
    '''
    Конветирует аудиофайлы из формата `mp3` в формат `wav`.

    Аргументы:
    ::  call_id_or_link -- идентификатор аудиофайла из директории `mp3`
        или ссылка на mp3-файл
    '''
    if call_id_or_uri[:4] == 'http':
        call_id = exstract_call_ids([call_id_or_uri])[0]
    else:
        call_id = call_id_or_uri
        call_id_or_uri = f'{os.path.join(trial_name, source, call_id)}.{source}'
    data, sample_rate = torchaudio.load(call_id_or_uri)
    if mono:
        data = data.mean(0, True)
    torchaudio.save(
        f'{os.path.join(trial_name, target, call_id)}.{target}',
        data,
        sample_rate,
        format=target,
        bits_per_sample=16
        )

### process

In [ ]:
for call_id in tqdm(get_call_ids('mp3')):
    converte(call_id)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
for link in tqdm(read_links()):
    converte(link, mono=True)

  0%|          | 0/19 [00:00<?, ?it/s]

## Speech Activity Detection
Определение периодов речевой активности для последующей сегментации.

### Defs

In [ ]:
def join_timelines(line: pd.DataFrame, min_duration_on: float=15.0):
    '''
    Соединение коротких периодов речевой активности
    для гарантии попадания в фрагмент разговора речи обоих собеседников.

    Аргументы:
    ::  line -- датафрейм с периодами речевой активности
    ::  min_duration_on -- минимальная продолжительеность речевой активности

    Возвращает датафрейм объединёнными периодами речевой активности.
    '''
    
    def check_durations(index: int) -> bool:
        '''
        Проверка длительности соседних периодов речевой активности.

        Аргументы:
        ::  index -- индекс последнего из пары проверяемых периодов

        Возвращает истину, если необходимо их слияние.
        '''
        return any(
            [line.duration_on.loc[index-i] < min_duration_on for i in (0, 1)]
        )

    line['duration_on'] = line.end - line.start
    line['duration_off'] = line.start - line.end.shift(fill_value=0)
    by = np.arange(len(line))
    agg_dict = {'start': 'first', 'end': 'last'}
    for index in line.duration_off[1:].sort_values().index.to_list():
        if check_durations(index):
            by[index-1] = index
            return line.groupby(by, as_index=False).agg(agg_dict)
    return None


def voice_activity_detection(
    model: object,
    ids: list=None
    ) -> dict:
    '''
    Определение периодов речевой активности для последующей сегментации.

    Аргументы:
    ::  model -- модель определяющая речевую активность
    ::  idx -- список идентификаторов аудиофайлов

    Возвращает словарь датафреймов с периодами речевой активности.
    '''
    if ids is None:
        ids = call_ids
    binarize = Binarize(
        offset=0.4,
        onset=0.4,
        log_scale=True, 
        min_duration_off=0.0,
        min_duration_on=1.0
        )
    lines = {}
    for call_id in tqdm(ids):
        path = os.path.join(trial_name, 'wav', call_id)
        lines[call_id] = pd.DataFrame(
            binarize.apply( 
                model({'audio': f'{path}.wav'}),
                dimension=1
                )
            )
        while True:
            new_line = join_timelines(lines[call_id])    
            if new_line is None:        
                break
            lines[call_id] = new_line
    return lines

### process

In [ ]:
%%capture
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')  # загрузка модели

In [ ]:
call_ids = get_call_ids(['links.txt'])
len(call_ids)

19

In [ ]:
lines = voice_activity_detection(sad, call_ids)

  0%|          | 0/1 [00:00<?, ?it/s]

## Segmentation
Сегментация исходных аудио-файлов на фрагменты длины подходящей для последующей сепарации каналов.

In [ ]:
def split_wave(
    wave: Wave,
    line: pd.DataFrame,
    call_id: str=None,
    save: bool=True
    ) -> Wave:
    '''
    Сегментация аудиозаписей.

    Аргументы:
    ::  wave -- именованный кортеж тензора и частоты дискретизации аудиозаписи
    ::  line: -- датафрейм с периодами речевой активности
    ::  call_id -- идентификатор аудиозаписи,
    ::  save: -- сохранять сегменты, если True

    Возвращает именованный кортеж списка сегментов и частоты дискретизации.
    '''
    channels = wave.data.shape[0]
    index = np.append(
        line[START_END].values
        .flatten()[1:-1].round(2).reshape(-1, 2).mean(1) 
        * wave.rate,
        wave.data.shape[-1]
        ).astype(int)
    segments = torch.split(
        wave.data,
        list(np.hstack([index[0], np.diff(index)])),
        dim=1
        )
    if save:
        path = f'{trial_name}/segments/{channels}'
        os.mkdir(os.path.join(path, call_id))
        for segment_id, segment in enumerate(segments):
            torchaudio.save( 
                f'{path}/{call_id}/{segment_id}.wav',
                segment,
                wave.rate,
                format='wav',
                bits_per_sample=16
                )
    return Wave(segments, wave.rate)

In [ ]:
waves = load_waves(call_ids)
segments = {
    call_id: split_wave(waves[call_id], lines[call_id], call_id)
    for call_id in tqdm(call_ids)
    }

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## Separation
Разделение одноканальных аудиозаписей на два канала для последующей диаризации.

In [ ]:
def recursive_separate(segment: torch.Tensor, max_length: int) -> torch.Tensor:
    '''
    Рекурсивная сепарация каналов.

    Аргументы:
    ::  segment - тензор фрагмента аудиозаписи
    ::  max_length - максимальная длина тензора,
        позволяющая проводить сепарацию без дополнительной сегментации

    Возвращает двуканальный тензор фрагмента.
    '''
    length = segment.shape[-1]
    if length > max_length:
        n = length // 2
        return torch.cat(
            [recursive_separate(segment[:, :n]), recursive_separate(segment[:, n:])],
            dim=1
        )
    return normalize(separator.separate_batch(segment).squeeze().T.cpu())


def separate(
    segments: list,
    sample_rate: int,
    call_id: str,
    max_length: int=500000
    ) -> None:
    '''
    Сепарация каналов и сохранение двухканальных аудиофайлов.
    '''
    path = f'{trial_name}/segments/2'
    if call_id not in os.listdir(path):
        os.mkdir(os.path.join(path, call_id))
    for segment_id, segment in enumerate(segments):
        separated = recursive_separate(segment, max_length)            
        torchaudio.save(
            f'{path}/{call_id}/{segment_id}.wav',
            separated, 
            sample_rate,
            format='wav',
            bits_per_sample=16
            )

### process

In [ ]:
%%capture
separator = SepformerSeparation.from_hparams(
    source='speechbrain/sepformer-wsj02mix',
    savedir='pretrained_models/sepformer-wsj02mix',
    run_opts={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    )

In [ ]:
for call_id in tqdm(call_ids):
    wave = load_segments(call_id, 1)
    if wave is not None:
        separate(*wave, call_id)

  0%|          | 0/1 [00:00<?, ?it/s]

## Speech to Text
Распознавание речи с помощью Yandex SpeechKit.

В рабочем катадлге должен быть файл keys.txt вида:


```
api_key <string>
aws_access_key_id <string>
aws_secret_access_key <string>
```

Инструкции для получения данных ключей находятся по ссылкам:

*   https://cloud.yandex.ru/docs/iam/operations/api-key/create
*   https://cloud.yandex.ru/docs/iam/operations/sa/create-access-key

Отправка сегментов в облако Яндекса для последующего распознавания.
Инструкция по созданию хранилища: https://cloud.yandex.ru/docs/storage/operations/buckets/create

### defs

In [ ]:
def clear_bucket(call_id, bucket):
    return s3.delete_objects( 
        Bucket=bucket,
        Delete={'Objects': [{'Key': call_id}]}
        )


def read_access(path: str='cloud_access.txt') -> pd.Series:
    '''
    Загрузка ключей от Yandex Cloud.

    Аргументы:
    ::  path -- путь к файлу с ключами

    Возвращает серию строк.
    '''
    with open(path) as f:
        access = pd.Series(
            {k: v for k, v in [line.strip().split() for line in f.readlines()]}
        )
    return access


def recognize(
    call_id: str,
    segment_id: str,
    bucket: str,
    api_key: str,
    channels: str,
    sample_rate: int,
    ):
    '''
    Распознавание речи и сохранение результата в формате JSON.
    '''
    path = os.path.join(access.endpoint, bucket, call_id, segment_id)
    upload_to_bucket(call_id, segment_id, channels)
    headers = {'Authorization': f'Api-Key {access.api_key}'}
    body = {
        'config': {
            'specification': {
                'languageCode': 'ru-RU',
                'audioEncoding': 'LINEAR16_PCM',
                'sampleRateHertz': sample_rate,
                'audioChannelCount': channels
            }
        },
        'audio': {'uri': f'{path}.wav'}
    }
    operation_id = requests.post(
        access.url,
        json=body,
        headers=headers
        ).json()['id']
    url = os.path.join(access.operations, operation_id)
    while True:
        time.sleep(2)
        response = requests.get(url, headers=headers).json()
        if response['done']:
            break
    clear_bucket(call_id, bucket)
    return response


def upload_to_bucket(call_id, segment_id, channels) -> None:
    '''
    Загрузка аудиофайла в Yandex Storage.
    '''
    path = os.path.join(trial_name, 'segments', channels, call_id)
    s3.upload_file( 
        f'{os.path.join(path, segment_id)}.wav',
        bucket,
        f'{os.path.join(call_id, segment_id)}.wav'
        )

### process

In [ ]:
access = read_access()
bucket = input('Введите название хранилища: ')
session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url=access.endpoint,
    aws_access_key_id=access.aws_access_key_id,
    aws_secret_access_key=access.aws_secret_access_key
)

Введите название хранилища: psycoder


In [ ]:
recognized = {}
for call_id in tqdm(call_ids):
    params = get_wave_params(call_id)
    channels = str(params.channels)
    path = os.path.join(trial_name, 'segments', channels, call_id)
    segment_ids = range(len(os.listdir(path)))
    recognized[call_id] = [
        recognize(
            call_id,
            str(segment_id),
            bucket,
            access.api_key,
            channels,
            params.sample_rate
            )
        for segment_id in segment_ids        
    ]
    path = os.path.join(trial_name, 'words', call_id)
    with open(f'{path}.json', 'w') as f:
        json.dump(recognized[call_id], f)  

  0%|          | 0/1 [00:00<?, ?it/s]

## Word Annotation
Создание датафреймов пословных аннотаций.

### defs

In [ ]:
def get_words(recognized, channels, start_time):
    '''
    Упаковка результатов распознавания речи в датафрейм.
    '''
    chunks = []
    for chunk in recognized['response']['chunks']:
        df = pd.DataFrame(chunk['alternatives'][0]['words'])
        if channels == 2:
            df['speaker'] = ['operator', 'client'][int(chunk['channelTag'])-1]
        chunks.append(df)
    rename_dict = {'startTime': 'segment_start', 'endTime': 'segment_end'}
    words = pd.concat(chunks, ignore_index=True).rename(columns=rename_dict)
    periods = words[['segment_start', 'segment_end']].applymap(
        lambda x: float(x[:-1])
    )
    words.loc[:, ['segment_start', 'segment_end']] = periods
    words[START_END] = periods + start_time
    words.sort_values(START_END, ignore_index=True, inplace=True)
    return words.drop('confidence', 1)


def load_recognized(ids: list=None) -> dict:
    '''
    Загрузка результатов распознавания речи.
    '''
    if ids is None:
        ids = call_ids
    recognized = {}
    for call_id in tqdm(ids):
        with open(f'{trial_name}/words/{call_id}.json', 'r') as f:
            recognized[call_id] = json.load(f)
    return recognized


def make_word_annotation(call_id: str) -> pd.DataFrame:
    '''
    Создание датафрейма пословной аннотации аудиозаписи.
    '''

    start_time = 0
    params = get_wave_params(call_id)
    annotations = []
    for segment_id, segment in enumerate(recognized[call_id]):
        df = get_words(segment, params.channels, start_time)
        annotations.append(df)
        if 'segments' in globals():
            length = segments[call_id].data[segment_id].shape[1]
        else:
            length = load_segments(
                call_id,
                params.channels,
                [segment_id]
            ).data[0].shape[1]
        start_time += round(length / params.sample_rate, 2) 
    return pd.concat(
        annotations,
        keys=range(len(annotations)),
        names=['segment_id']
        ).reset_index().drop('level_1', 1)


def save_word_annotation(call_id: str) -> None:
    '''
    Сохранение датафрейма пословной аннотации аудиозаписи
    в форматах `csv` и `txt` (для импорта в Audacity).
    '''
    path = f'{trial_name}/words/{call_id}'
    words[call_id].to_csv(f'{path}.csv', index=False)
    if 'speaker' in words[call_id].columns:
        mask = words[call_id].speaker == 'client'
        labels = words[call_id].loc[:, ['start', 'end', 'word']].dropna()        
        labels.loc[mask, 'word'] = words[call_id].loc[mask, 'word'].str.upper()
        labels.to_csv(
            f'{path}.txt',
            sep='\t',
            header=False,
            index=False
            )

### process

In [ ]:
recognized = load_recognized(call_ids)
words = {}
for call_id in tqdm(call_ids):
    words[call_id] = make_word_annotation(call_id)
    save_word_annotation(call_id)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## Speaker Recognition
Распознавание принадлежности слов к клиенту или оператору в одноканальных аудиозаписях.

### defs

In [ ]:
def add_speakers(
    call_id: str,
    diarization_rate: int=100,
    kernel_size: int=2000
    ) -> None:
    '''
    Добавление во фрейм пословной аннотации роли говорящего.
    '''
    segments = load_segments(call_id, 2)
    resample = Resample(segments.rate, diarization_rate)
    avg = AvgPool1d(kernel_size, 1)
    pad = ReplicationPad1d(kernel_size//2)
    data = torch.cat(segments.data, 1).abs().unsqueeze(0)   
    volumes = normalize(resample(avg(pad(data)).squeeze())).numpy()
    end_time = words[call_id].end.iloc[-1]
    diarization = make_diarization(volumes, diarization_rate, end_time)
    words[call_id]['speaker'] = words[call_id][START_END].apply(
        get_speaker,
        diarization=diarization,
        axis=1
    )


def make_diarization(volumes, diarization_rate, end_time, save=True):
    '''
    Создание датафрейма оценок предлежности слов к оператору или клиенту.
    '''
    columns = ['operator', 'client']
    diarization = pd.DataFrame(
        data=volumes.T,
        index=(np.arange(volumes.shape[1]) / diarization_rate).round(2),
        columns=columns
    )
    while diarization.index[-1] < end_time:
        index = diarization.index[-1]
        diarization.loc[round(index + 0.01, 2), columns] = (
            diarization.loc[index, columns]
            )
    if save:
        diarization.to_csv(os.path.join(trial_name, 'diarizations', f'{call_id}.csv'))
    return diarization


def get_speaker(period, diarization, marginal=True):
    '''
    Определение говорящего в заданный период времени.
    '''    
    start, end = period.values.round(2)
    if marginal:
        return diarization.loc[[start, end]].mean().idxmax()
    return diarization.loc[start:end].mean().idxmax()
    # return (diarization.loc[[start, end]].mean() + diarization.loc[start:end].mean()).idxmax()

### process

In [ ]:
words = load_word_annotations(call_ids)
for call_id in tqdm(call_ids):
    if get_wave_params(call_id).channels == 1:
        add_speakers(call_id)
    save_word_annotation(call_id)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## Dialog Annotation
Представление пословной аннотации в виде диалога.

### defs

In [ ]:
def bridge_gaps(df, max_gap=0.1):
    ids = (df.start - df.end.shift(fill_value=0) > max_gap).cumsum()
    return group_by_ids(df, ids)


def drop_echo(words):
    operator = words.query('speaker == "operator"')
    client = words.query('speaker == "client"')
    operator = operator.loc[~operator.apply(is_echo, 1, words=client)]
    client = client.loc[~client.apply(is_echo, 1, words=operator)]
    return pd.concat(
        [operator, client],
        ignore_index=True
        ).sort_values(START_END)


def group_by_ids(df, ids):
    agg_dict = {
        'start': 'first',
        'end': 'last',
        'word': ' '.join,
        'speaker': 'last'
        }
    return df.groupby(ids).agg(agg_dict)


def format_time(seconds: float) -> str:
    '''
    Форматирование временных меток.
    '''
    return str(pd.Timedelta(int(seconds * 1000), unit='ms')).split()[-1][:12]


def is_echo(label, words, delay=1, delta=0.5):
    words = (
        words.set_index('start')
        .loc[label.start-delay:label.start]
        .reset_index().query('word == @label.word')[START_END]
        )    
    if len(words):
        start, end = words.values[0]
        return abs(label.end - label.start - end + start) < delta
    return False


def make_dialog(words, humanly=True):
    '''
    Создание датафрейма диалога.
    '''
    dialog = group_by_ids(
        words,
        (words.speaker == 'client').diff().fillna(0).cumsum()
    )
    dialog = pd.concat(
        [
            bridge_gaps(dialog.query('speaker == "operator"')),
            bridge_gaps(dialog.query('speaker == "client"'))
        ]
    ).sort_values(START_END)
    dialog['speech_id'] = np.arange(len(dialog))
    if humanly:
        dialog[START_END] = dialog[START_END].applymap(format_time)
    return dialog.rename(columns={'word': 'speech'}).set_index('speech_id')

### process

In [ ]:
call_ids = get_call_ids(['links.txt'])

In [ ]:
call_ids = ['mix_13054_16e013__2021_08_26__17_45_09_037']

In [ ]:
words = load_word_annotations(call_ids)

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
dialogs = {}
for call_id in tqdm(call_ids):
    dialogs[call_id] = make_dialog(drop_echo(words[call_id]))
    dialogs[call_id].to_csv(f'{trial_name}/dialogs/{call_id}.csv', index=False)

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
dialog = dialogs[call_id]
dialog

,start,end,speech,speaker
speech_id,,,,
0,1.32,2.56,добрый день,operator
1,2.54,3.52,здравствуйте,client
2,4.45,14.85,данные я представляю компанию рольф официального дилера марки ниссан вы планируете покупку нового автомобиля нисан в...,operator
3,15.42,17.82,да звонила поняла на прошлой неделе,client
4,18.92,40.40,ну в таком случае мы можем вас проконсультировать мы хотели вам сообщить о том что сейчас у нас действует специальна...,operator
5,41.05,54.68,ну я разговаривала у меня был внук я хотела внуку купить ну сейчас даже не знаю он же уехал так нас никто не ответил...,client
6,45.62,46.40,угу,operator
7,53.01,53.81,да,operator
8,54.82,56.46,ну мы можем предложить,operator


In [ ]:
dialog = dialogs[call_id]
dialog

,start,end,speech,speaker
speaker,,,,
0,1.32,2.56,добрый день,operator
1,2.54,3.52,здравствуйте,client
2,4.45,14.85,данные я представляю компанию рольф официального дилера марки ниссан вы планируете покупку нового автомобиля нисан в...,operator
3,15.42,17.82,да звонила поняла на прошлой неделе,client
4,18.92,40.40,ну в таком случае мы можем вас проконсультировать мы хотели вам сообщить о том что сейчас у нас действует специальна...,operator
5,41.05,45.79,ну я разговаривала у меня был внук я хотела внуку купить ну,client
6,45.62,46.40,угу,operator
7,45.83,53.06,сейчас даже не знаю он же уехал так нас никто не ответил он уехал будет на выходных теперь есть что что бы вы,client
8,53.01,53.81,да,operator


In [ ]:
operator = dialog.query('speaker == "operator"')
client = dialog.query('speaker == "client"')

operator.start - operator.end.shift(fill_value=0) < 0.1

speaker
0     False
2     False
4     False
6     False
8     False
10    False
12     True
14     True
16    False
18     True
20     True
22    False
24    False
26    False
28    False
30    False
32    False
34     True
36     True
38    False
40    False
42     True
44     True
46     True
dtype: bool

speaker
0      1
2      2
4      3
6      4
8      5
10     6
12     6
14     6
16     7
18     7
20     7
22     8
24     9
26    10
28    11
30    12
32    13
34    13
36    13
38    14
40    15
42    15
44    15
46    15
dtype: int64

Разультаты распознавания в папке "dialogs".

# EOF